In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
#19년 20년 월별 데이터 생성
data2019 = {}
data2020 = {}

class subWayData:
    def __init__(self,dataY):
        pass
    def create_data(self):
        pass
    

for indx,csvF in enumerate(os.listdir('./seoul_subway_data/2019')):
    #19년도 데이터 안에 이미 데이터가 있으면 넘어가고 없으면 기본값으로 추가
    if not 'data' + csvF[-10:-4] in data2019.keys():
        data2019.setdefault('data' + csvF[-10:-4])
    else:
        continue
        
    try:
        data = pd.read_csv('./seoul_subway_data/2019/' + csvF,encoding = 'UTF-8')
    except:
        data = pd.read_csv('./seoul_subway_data/2019/' + csvF,encoding = 'EUC-KR')
    data2019['data' + csvF[-10:-4]] = data

for indx,csvF in enumerate(os.listdir('./seoul_subway_data/2020')):
    #20년도 데이터 안에 이미 데이터가 있으면 넘어가고 없으면 기본값으로 추가
    if not 'data' + csvF[-10:-4] in data2020.keys():
        data2020.setdefault('data' + csvF[-10:-4])
    else:
        continue
        
    #인코딩 부분에서 문제가 있는 경우가 있어서 기본값 utf-8로 처리에 에러가 있을경우 euc-kr로 대체
    try:
        data = pd.read_csv('./seoul_subway_data/2020/' + csvF,encoding = 'UTF-8')
    except:
        data = pd.read_csv('./seoul_subway_data/2020/' + csvF,encoding = 'EUC-KR')
    data2020['data' + csvF[-10:-4]] = data

#2019년 데이터에서 불필요한 데이터 조정
for dataY in data2019.keys():
    if '역ID' in data2019[dataY].columns:
        data2019[dataY].drop('등록일자',axis=1,inplace = True)
        data2019[dataY].rename(columns = {'하차총승객수' : '등록일자'},inplace = True)
        data2019[dataY].rename(columns = {'승차총승객수' : '하차총승객수'},inplace = True)
        data2019[dataY].rename(columns = {'역명' : '승차총승객수'},inplace = True)
        data2019[dataY].rename(columns = {'역ID' : '역명'},inplace = True)
    else:
        data2019[dataY].dropna(inplace=True)

#2020년 데이터에서 불필요한 데이터 조정
for dataY in data2020.keys():
    if '역ID' in data2020[dataY].columns:
        data2020[dataY].drop('등록일자',axis=1,inplace = True)
        data2020[dataY].rename(columns = {'하차총승객수' : '등록일자'},inplace = True)
        data2020[dataY].rename(columns = {'승차총승객수' : '하차총승객수'},inplace = True)
        data2020[dataY].rename(columns = {'역명' : '승차총승객수'},inplace = True)
        data2020[dataY].rename(columns = {'역ID' : '역명'},inplace = True)
    else:
        data2020[dataY].dropna(inplace=True)

<h1>데이터 정리 최종</h1>

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import os, csv

option = webdriver.ChromeOptions()
option.add_argument('headless')

driver = webdriver.Chrome('/Users/sanengr/Downloads/chromedriver',options=option)
driver.get('https://gits.gg.go.kr/gtdb/web/trafficDb/railRoad/TransitSWPass.do')
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

time.sleep(3)

def setRadio():
    driver.find_element_by_xpath('//*[@id="radio1"]').click()

def setSelect1(sel1):
    driver.find_element_by_xpath("//select[@name='select1']/option[text()='%s']" %sel1).click()
    
def setSelect2(sel2):
    driver.find_element_by_xpath("//select[@name='select2']/option[text()='%s']" %sel2).click()

#soup로 고정된 html 값에서 radio1 버튼 찾아와서 클릭하기
setRadio()

#rad 버튼 클릭후 동적으로 변경된 웹페이지에서 select1아이디 값 가져오기
select1 = driver.find_element_by_id('select1')
t_sel1 = select1.text.split('\n')
sel1 = []
for i,t in enumerate(t_sel1):
    if not t.strip() == '':
        sel1.append(t.strip())

#시 - 역명 넣을 csv파일 생상(기존에 있는건 삭제)
if os.path.exists('./seoul_subway_data/구_역명.csv'):
    os.remove('./seoul_subway_data/구_역명.csv')

with open('./seoul_subway_data/구_역명.csv','w') as csvFile:
    writer = csv.DictWriter(csvFile,fieldnames=['구','역명'])    
    writer.writeheader()
    
#sel1 값을 넣은 후 sel2값 넣고 검색 후 list값들을 가져온 다음 데이터 정리
for i in sel1:
    setSelect1(i)
    
    sel2 = driver.find_element_by_id('select2')
    t_sel2 = sel2.text.split('\n')
    for a in range(len(t_sel2)):
        t_sel2[a] = t_sel2[a].strip()
        #전체 선택 혹은 비어있는 경우는 넘어가기 (행정 구역만 필요하니까)
        if '전체선택' == t_sel2[a] or '' == t_sel2[a]:
            continue
        
        setSelect2(t_sel2[a])
        driver.find_element_by_xpath('//*[@id = "search"]').click()
        li_loc = (driver.find_element_by_id('selList')).text.split('\n')

        for i in li_loc:
            #가져온 list값은 매봉(1호선) 이런식으로 돼있으므로 ()를 기준으로 역이름과 호선명 나눠주기
            st_idx = i.find('(') + 1
            f_idx = i.find(')')
            t_sel2[a] = t_sel2[a].replace('_','')
            #csv파일에 바로 넣기
            with open('./seoul_subway_data/구_역명.csv','a') as csvFile:
                writer = csv.DictWriter(csvFile,fieldnames=['구','역명'])
                writer.writerow({'구':t_sel2[a],'역명':i[:st_idx-1]})

gu_sta_name = pd.read_csv('./seoul_subway_data/구_역명.csv')

#2019년 데이터에 구 추가
for dataY in data2019.keys():
    if not '구' in data2019[dataY].columns:    
        data2019[dataY] = pd.merge(data2019[dataY],gu_sta_name,on='역명',how = 'inner')

        if True in data2019[dataY].duplicated():
            data2019[dataY] = data2019[dataY].drop_duplicates()
            data2019[dataY].reset_index(inplace=True)
        if 'index' in data2019[dataY].columns:
            data2019[dataY].drop(columns='index',inplace=True)
    
#2020년 데이터에 구 추가    
for dataY in data2020.keys():
    if not '구' in data2020[dataY].columns:    
        data2020[dataY] = pd.merge(data2020[dataY],gu_sta_name,on='역명',how = 'inner')

        if True in data2020[dataY].duplicated():
            data2020[dataY] = data2020[dataY].drop_duplicates()
            data2020[dataY].reset_index(inplace=True)
        if 'index' in data2020[dataY].columns:
            data2020[dataY].drop(columns='index',inplace=True)

In [5]:
# import json
# import folium

# geo_path = '/Users/sanengr/Downloads/python/tmp/southkorea-maps/kostat/2018/json/skorea-municipalities-2018-geo.json'
# geo_str = json.load(open(geo_path, encoding='utf-8'))

# map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
#                 tiles='stamentoner')
# map.choropleth(geo_data=geo_str)

# print(geo_str.keys())
# print(geo_str['features'][0].keys())


<h1>그냥 바로 해버리면 전국에 칠해지니까 서울지역으로 json 범위 축소 </h1><br><p>근데 왜 한번에 안되고 여러번에 거쳐야 되는지는 모르겠음...</p>

In [6]:
import json
import folium

geo_path = './map/skorea-municipalities-2018-geo.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

#json 데이터 안다듬어주면 전국단위로 난리남
print('json 처리 전 지역 수 : ',len(geo_str['features']))
min_len_geoStr = False
k = [-1,-2]
#서울 지역 코드 값 = 1로 시작
#왜인지는 모르겠는데 한번에 처리가 안되서 이전 카운트 개수하고 현재 카운트 개수에 차이가 없을 때 break
while min_len_geoStr == False:
    if k[0] == k[1]:
        min_len_geoStr = True
    k[0] = len(geo_str['features'])
    for i,t in enumerate(geo_str['features']):
        if not t['properties']['code'][0] == '1':
            del geo_str['features'][i]
    k[1] = len(geo_str['features'])        

print('json 처리 후 지역 수 : ',len(geo_str['features']))

class mapping:
    global geo_str
    
    def __init__(self,dataY):
        self.map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                    tiles='stamentoner')
        
        self.dataY = dataY


    def drawMap(self):
        #맵 그려주기
        folium.Choropleth(
            geo_data=geo_str,
            data=self.dataY, 
            columns=['구','승차총승객수'], 
            key_on='feature.properties.name',
            fill_color='PuRd',
            legend_name='지하철 유동인구 수'
        ).add_to(self.map)

    def saveMap(self,YM):
        YM = str(YM)
        
        #쥬피터에서 보여주면 한글 깨져서 동일 경로 map 디렉토리에 해당 맵을 만들어줌
        if os.path.exists('./map'):
            if os.path.exists('./map/{}.html'.format(YM)):
                os.remove('./map/{}.html'.format(YM))
        else:
            os.makedirs('./map')
        self.map.save('./map/{}.html'.format(YM))
        
map2019 = mapping(data2019)
map2019.drawMap()
map2019.saveMap(2019)

map2020 = mapping(data2020)
map2020.drawMap()
map2020.saveMap(2020)


json 처리 전 지역 수 :  250
json 처리 후 지역 수 :  25


ValueError: cannot copy sequence with size 13901 to array axis with dimension 7

In [ ]:
# a = data2020[['노선명','역명','승차총승객수','하차총승객수','구']][data2020['승차총승객수'] - data2020['하차총승객수'] > 3000]
# a.shape

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
#19년 20년 월별 데이터 생성
data2019 = {}
data2020 = {}

class subWayData:
    def __init__(self,year):
        self.year = year
    
    def create_data(self):
        pass
    
for indx,csvF in enumerate(os.listdir('./seoul_subway_data/{}'.format(year))):
    #19년도 데이터 안에 이미 데이터가 있으면 넘어가고 없으면 기본값으로 추가
    if not 'data' + csvF[-10:-4] in data2019.keys():
        data2019.setdefault('data' + csvF[-10:-4])
    else:
        continue
        
    try:
        data = pd.read_csv('./seoul_subway_data/2019/' + csvF,encoding = 'UTF-8')
    except:
        data = pd.read_csv('./seoul_subway_data/2019/' + csvF,encoding = 'EUC-KR')
    data2019['data' + csvF[-10:-4]] = data

for indx,csvF in enumerate(os.listdir('./seoul_subway_data/2020')):
    #20년도 데이터 안에 이미 데이터가 있으면 넘어가고 없으면 기본값으로 추가
    if not 'data' + csvF[-10:-4] in data2020.keys():
        data2020.setdefault('data' + csvF[-10:-4])
    else:
        continue
        
    #인코딩 부분에서 문제가 있는 경우가 있어서 기본값 utf-8로 처리에 에러가 있을경우 euc-kr로 대체
    try:
        data = pd.read_csv('./seoul_subway_data/2020/' + csvF,encoding = 'UTF-8')
    except:
        data = pd.read_csv('./seoul_subway_data/2020/' + csvF,encoding = 'EUC-KR')
    data2020['data' + csvF[-10:-4]] = data

#2019년 데이터에서 불필요한 데이터 조정
for dataY in data2019.keys():
    if '역ID' in data2019[dataY].columns:
        data2019[dataY].drop('등록일자',axis=1,inplace = True)
        data2019[dataY].rename(columns = {'하차총승객수' : '등록일자'},inplace = True)
        data2019[dataY].rename(columns = {'승차총승객수' : '하차총승객수'},inplace = True)
        data2019[dataY].rename(columns = {'역명' : '승차총승객수'},inplace = True)
        data2019[dataY].rename(columns = {'역ID' : '역명'},inplace = True)
    else:
        data2019[dataY].dropna(inplace=True)

#2020년 데이터에서 불필요한 데이터 조정
for dataY in data2020.keys():
    if '역ID' in data2020[dataY].columns:
        data2020[dataY].drop('등록일자',axis=1,inplace = True)
        data2020[dataY].rename(columns = {'하차총승객수' : '등록일자'},inplace = True)
        data2020[dataY].rename(columns = {'승차총승객수' : '하차총승객수'},inplace = True)
        data2020[dataY].rename(columns = {'역명' : '승차총승객수'},inplace = True)
        data2020[dataY].rename(columns = {'역ID' : '역명'},inplace = True)
    else:
        data2020[dataY].dropna(inplace=True)

a
a
a
a
a
a
a
a
a
a
a
a
